In [ ]:
# import the basics
!pip install nb_black > /dev/null
%load_ext lab_black
import pandas as pd
import numpy as np
import sys
import os

In [ ]:
# import graphing packages
import matplotlib.pylab as plt

%matplotlib inline
import seaborn as sns

In [ ]:
# imports for run_bagged_validation
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn.metrics import f1_score, log_loss, precision_score, recall_score
import lightgbm as lgb
from tqdm.notebook import tqdm

In [ ]:
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics.pairwise import euclidean_distances
import scipy
import scipy.sparse
from sklearn import preprocessing
from scipy.sparse.linalg import expm
from sklearn.metrics.pairwise import euclidean_distances

In [ ]:
!pip install gseapy
import gseapy as gp
from gseapy.plot import barplot, dotplot

# Loading DESeq2 Datasets

In [ ]:
demographics = pd.read_csv(
    "/kaggle/input/end-als/end-als/clinical-data/filtered-metadata/metadata/clinical/Demographics.csv"
)
demographics.to_csv("/kaggle/working/demographics.csv")
alsfrs_scores = pd.read_csv(
    "/kaggle/input/end-als/end-als/clinical-data/filtered-metadata/metadata/clinical/ALSFRS_R.csv"
)
alsfrs_scores.to_csv("/kaggle/working/alsfrs_scores.csv")
bulbar_vs_limb = pd.read_csv(
    "/kaggle/input/end-als/end-als/transcriptomics-data/DESeq2/bulbar_vs_limb.csv"
)
bulbar_vs_limb.to_csv("/kaggle/working/bulbar_vs_limb.csv")
ctrl_vs_case = pd.read_csv(
    "/kaggle/input/end-als/end-als/transcriptomics-data/DESeq2/ctrl_vs_case.csv"
)
ctrl_vs_case.to_csv("/kaggle/working/ctrl_vs_case.csv")
median_low_vs_high = pd.read_csv(
    "/kaggle/input/end-als/end-als/transcriptomics-data/DESeq2/median_low_vs_high.csv"
)
median_low_vs_high.to_csv("/kaggle/working/median_low_vs_high.csv")
example_transcriptomics_3counts_data = pd.read_csv(
    "/kaggle/input/end-als/end-als/transcriptomics-data/L3_counts/CASE-NEUZX521TKK/CASE-NEUZX521TKK-5793-T/CASE-NEUZX521TKK-5793-T_P85.exon.txt",
    delim_whitespace=True,
    skiprows=1,
    low_memory=False,
)
example_transcriptomics_3counts_data.to_csv("/kaggle/working/L3_counts.csv")

In [ ]:
biomart_path = "/kaggle/input/biomart-annotation/mart_export.txt"
mart_export = pd.read_csv(biomart_path)
mart_export.head()

In [ ]:
mart_export["Gene type"].unique()

In [ ]:
pseudo_genes = [
    "transcribed_processed_pseudogene",
    "polymorphic_pseudogene",
    "processed_pseudogene",
    "unprocessed_pseudogene",
    "pseudogene",
    "rRNA_pseudogene",
    "IG_V_pseudogene",
    "TR_V_pseudogene",
    "unitary_pseudogene",
    "IG_C_pseudogene",
    "IG_J_pseudogene",
    "transcribed_unitary_pseudogene",
    "TR_J_pseudogene",
    "translated_processed_pseudogene",
    "TR_J_pseudogene",
    "translated_unprocessed_pseudogene",
    "IG_pseudogene",
]

In [ ]:
pd.DataFrame(ctrl_vs_case.columns[2:28], columns=["gene id"])

In [ ]:
s = mart_export["Transcript stable ID"].map(
    mart_export.set_index("Transcript stable ID")["Gene stable ID"]
)
mart_export["Gene_name2"] = mart_export["Gene name"].mask(
    mart_export["Gene name"].isnull(), s
)

In [ ]:
pseudogenes_annotate = mart_export[mart_export["Gene type"].isin(pseudo_genes)]
pseudogenes = ctrl_vs_case.columns[
    ctrl_vs_case.columns.isin(pseudogenes_annotate["Gene_name2"])
]

In [ ]:
def fisher(data_df, target, num_features):
    df = pd.DataFrame()
    df["mean"] = data_df[data_df[target] == 1].mean()
    df["control"] = data_df[data_df[target] == 0].mean()
    df["std_case"] = data_df[data_df[target] == 1].std()
    df["std_control"] = data_df[data_df[target] == 0].std()

    df["diff_mean"] = (df["mean"] - df["control"]).abs()
    df["sum_std"] = (df["std_case"] + df["std_control"]).abs()
    df["fisher_coeff"] = df["diff_mean"] / df["sum_std"]
    df = df.sort_values(["fisher_coeff"], ascending=False)
    most_significant_features = df["fisher_coeff"].index[:num_features]
    return list(most_significant_features.drop(target))

In [ ]:
def construct_W(X, neighbour_size=4, t=1):
    n_samples, n_features = np.shape(X)
    S = kneighbors_graph(X, neighbour_size + 1, metric="euclidean")
    S = (-1 * (S * S)) / (2 * t * t)
    S = S.tocsc()
    S = expm(S)
    S = S.tocsr()
    bigger = np.transpose(S) > S
    S = S - S.multiply(bigger) + np.transpose(S).multiply(bigger)
    return S


def LaplacianScoreTopK(X, topK, columnlist, neighbour_size=4, t=1):
    W = construct_W(X, t=t, neighbour_size=neighbour_size)
    n_samples, n_features = np.shape(X)
    D = np.array(W.sum(axis=1))
    D = scipy.sparse.diags(np.transpose(D), [0])
    L = D - W.toarray()

    I = np.ones((n_samples, n_features))

    Xt = np.transpose(X)

    t1 = np.matmul(np.matmul(Xt, D.toarray()), I) / np.matmul(
        np.matmul(np.transpose(I), D.toarray()), I
    )
    t1 = t1[:, 0]
    t1 = np.tile(t1, (n_samples, 1))
    fr = X - t1

    fr_t = np.transpose(fr)
    Lr = np.matmul(np.matmul(fr_t, L), fr) / np.matmul(np.dot(fr_t, D.toarray()), fr)

    Lap_vals = np.diag(Lr)
    top_index = pd.Series(Lap_vals).sort_values().head(topK).index
    return list(columnlist[top_index])

In [ ]:
def get_split_feature_importance(clf, split_n):
    fi_df = pd.DataFrame(
        clf.feature_importances_,
        index=clf.feature_name_,
        columns=[f"importance_{split_n}"],
    )
    return fi_df

In [ ]:
def compute_metrics(y_val, pred_val, pred_probs_val):
    f1 = f1_score(y_val, pred_val)
    prec = precision_score(y_val, pred_val)
    rec = recall_score(y_val, pred_val)
    loglosscore = log_loss(y_val, pred_probs_val)
    print(
        f"F1 Score: {f1:0.4f} - Precision {prec:0.4f} - Recall {rec:0.4f} - LogLoss {loglosscore:0.4f}"
    )

In [ ]:
def run_bagged_validation_new(
    data,
    target_col,
    nsplits,
    lgb_params,
    gene_subset,
    test_size=0.1,
    random_state=529,
    method=None,
    topK=None,
    print_metrics=True,
):
    X = data[gene_subset].copy()
    y = data[target_col].copy()

    sss = StratifiedShuffleSplit(
        n_splits=nsplits, test_size=test_size, random_state=random_state
    )

    pred_val_probs_all = []
    pred_val_all = []
    y_val_all = []
    fis = []
    split_n = 0
    for tr_idx, val_idx in tqdm(sss.split(X, y), total=nsplits):
        y_tr = y.iloc[tr_idx]
        y_val = y.iloc[val_idx]
        if method == "Fisher" and (topK is not None):
            fisher_features = fisher(data.iloc[tr_idx], target_col, topK)
            X_tr = data.iloc[tr_idx][gene_subset + fisher_features]
            X_val = data.iloc[val_idx][gene_subset + fisher_features]
        elif method == "Laplacian" and (topK is not None):
            N = 10000
            TOP_N_HIGH_VARIANCE = (
                data.iloc[tr_idx]
                .drop(["Participant_ID", target_col], axis=1)
                .var()
                .sort_values(ascending=False)
                .head(N)
                .index.tolist()
            )
            data2 = data[TOP_N_HIGH_VARIANCE]
            laplacian_features = LaplacianScoreTopK(
                np.array(data2.iloc[tr_idx]),
                topK=topK,
                columnlist=data2.columns,
            )
            X_tr = data.iloc[tr_idx][gene_subset + laplacian_features]
            X_val = data.iloc[val_idx][gene_subset + laplacian_features]
        else:
            X_tr = X.iloc[tr_idx]
            X_val = X.iloc[val_idx]
        clf = lgb.LGBMClassifier(**lgb_params)
        clf.fit(X_tr, y_tr)
        pred_val_probs = clf.predict_proba(X_val)[:, 0]
        pred_val = clf.predict(X_val)
        # Store predictions for each split
        pred_val_probs_all.append(pred_val_probs)
        pred_val_all.append(pred_val)
        y_val_all.append(y_val)

        fis.append(get_split_feature_importance(clf, split_n))

        split_n += 1
    pred_val_all = np.concatenate(pred_val_all)
    pred_val_probs_all = np.concatenate(pred_val_probs_all)
    y_val_all = np.concatenate(y_val_all)

    results = pd.DataFrame(
        [y_val_all, pred_val_all, pred_val_probs_all],
        index=[target_col, "pred", "pred_probs"],
    ).T

    fis_all = pd.concat(fis, axis=1)
    if print_metrics:
        compute_metrics(y_val_all, pred_val_all, pred_val_probs_all)
    return results, fis_all

In [ ]:
lgb_params = {
    "num_leaves": 31,
    "max_depth": -1,
    "learning_rate": 0.1,
    "n_estimators": 10_000,
}
results_fisher, fis_all_fisher = run_bagged_validation_new(
    ctrl_vs_case.drop(columns=pseudogenes),
    gene_subset=[],
    target_col="CtrlVsCase_Classifier",
    method="Fisher",
    nsplits=10,
    lgb_params=lgb_params,
    topK=10,
)

In [ ]:
font = {"family": "normal", "size": 18}
plt.rc("font", **font)
feature_order_fisher = fis_all_fisher.mean(axis=1).sort_values(ascending=False).index
fig, ax = plt.subplots(figsize=(8, 8))
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["bottom"].set_visible(False)
ax.spines["left"].set_visible(False)
ax.set(xlabel="Fisher Scores", ylabel="Genes")
data = fis_all_fisher.loc[feature_order_fisher].T
values = {}
important_columns = []
for col in data.columns:
    if (data[col] > 1000).value_counts()[False] < 10:
        important_columns.append(col)
        values[col] = data[col].max()
data = data[important_columns]

g = sns.barplot(data=data, palette="pastel", orient="h", ax=ax, ci=None)

In [ ]:
lgb_params = {
    "num_leaves": 31,
    "max_depth": -1,
    "learning_rate": 0.1,
    "n_estimators": 10_000,
}
results_lap, fis_all_lap = run_bagged_validation_new(
    ctrl_vs_case.drop(columns=pseudogenes),
    gene_subset=[],
    method="Laplacian",
    target_col="CtrlVsCase_Classifier",
    nsplits=10,
    lgb_params=lgb_params,
    topK=10,
)

In [ ]:
feature_order_lap = fis_all_lap.mean(axis=1).sort_values(ascending=False).index
fig, ax = plt.subplots(figsize=(10, 20))
sns.barplot(
    data=fis_all_lap.loc[feature_order_lap].T,
    palette="pastel",
    orient="h",
    ax=ax,
    ci=None,
)